In [10]:
# crop the image to fit 1 ha grid

import rasterio
from rasterio.windows import Window

def tm_to_pixel(tm_coords, transform):
    """ Convert TM2 coordinates to pixel coordinates. """
    # Unpack the TM coordinates
    tm_x, tm_y = tm_coords
    
    # Transform the coordinates using the raster's transform
    pixel_x, pixel_y = ~transform * (tm_x, tm_y)  # Inverse transformation to get pixel coordinates
    return int(pixel_x), int(pixel_y)

def crop_tiff(input_file, output_file, top_left_original, top_left_new):
    with rasterio.open(input_file) as src:
        # Get image dimensions and transform
        img_width = src.width
        img_height = src.height
        transform = src.transform

        print(img_width, img_height)
        
        # Calculate bottom-right corner of the original image
        bottom_right_x_original = top_left_original[0] + img_width
        bottom_right_y_original = top_left_original[1] + img_height
        
        # Convert TM2 coordinates to pixel coordinates
        top_left_new_pixel = tm_to_pixel(top_left_new, transform)

        # Define the cropping area (left, upper, right, lower)
        box = (
            top_left_new_pixel[0],  # left
            top_left_new_pixel[1],  # upper
            bottom_right_x_original, # right
            bottom_right_y_original   # lower
        )

        print(box)
        
        # Ensure the box coordinates are valid
        box = (
            max(box[0], 0),  # left
            max(box[1], 0),  # upper
            min(box[2], img_width),  # right
            min(box[3], img_height)   # lower
        )
        
        # Read the data within the defined bounds and crop
        width = bottom_right_x_original-top_left_new_pixel[0]
        height = bottom_right_y_original-top_left_new_pixel[1]
        window = Window(top_left_new_pixel[0], top_left_new_pixel[1], width, height)
        transform = src.window_transform(window)
        cropped_image = src.read(window=window)

        print(window)
          
        # Update metadata for the cropped image
        metadata = src.meta.copy()
        metadata.update({
            'height': height,
            'width': width,
            'transform': transform
        })
        
        # Save the cropped image
        # with rasterio.open(output_file, 'w', **metadata) as dst:
        #     dst.write(cropped_image)

# Example usage
input_tiff = 'h:\\Yehmh\\ZF\\202405\\merge_clip.tif'  # Path to your input TIFF file
output_tiff = 'cropped_image.tiff'  # Path for saving the cropped image
top_left_original = (0, 0)  # Coordinates of the original image's top-left corner
top_left_new = (298509.470395326265134, 2633004.086879610549659)  # Your specified new top-left TM2 coordinate

crop_tiff(input_tiff, output_tiff, top_left_original, top_left_new)


67748 52664
(1703, 1480, 67748, 52664)
Window(col_off=1703, row_off=1480, width=66045, height=51184)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [6]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
from shapely.geometry import box

tiff_file = 'h:\\Yehmh\\DNDF\\202404_DNDF\\DNDF7Band.tif'
grid_shapefile = "h:\\Yehmh\\DNDF\\202404_DNDF\\spectral_diversity\\north_grid.shp"
output_shapefile = 'h:\\Yehmh\\DNDF\\202404_DNDF\\spectral_diversity\\alpha_diversity_filtered4.shp'

# Load the grid shapefile
grid = gpd.read_file(grid_shapefile)

# Load the TIFF file
with rasterio.open(tiff_file) as src:
    # Get the bounding box of the TIFF file and convert it to a Polygon
    tiff_bounds = src.bounds
    tiff_polygon = box(tiff_bounds.left, tiff_bounds.bottom, tiff_bounds.right, tiff_bounds.top)

    grid_data = []

    for _, block in grid.iterrows():
        # Check if the block is within the TIFF file bounds
        if block.geometry.intersects(tiff_polygon):
            # Mask the TIFF file with the grid block
            masked_tile, transform = rasterio.mask.mask(src, [block.geometry], crop=True)

            # Calculate squared deviations
            squared_deviation = (masked_tile - masked_tile.mean(axis=(1, 2), keepdims=True))**2
            
            # Calculate the sum of squares for each band
            sum_of_squares = np.sum(squared_deviation, axis=(1, 2))
            
            # Divide the sum by the number of pixels to get mean squared deviation
            mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))
            
            # Calculate the total sum of squares across all bands
            total_sum_of_squares = np.sum(mean_squared_deviation)
            
            # Calculate FCSD (each band's mean squared deviation divided by total sum of squares)
            fcsd = mean_squared_deviation / total_sum_of_squares
            
            # Append the grid cell's data to the list
            grid_data.append({
                'block_id': block['FID'],  # or block's coordinates
                'geometry': block.geometry,
                'mean_squared_deviation_ir': mean_squared_deviation[0],
                'mean_squared_deviation_red_edge': mean_squared_deviation[1],
                'mean_squared_deviation_red': mean_squared_deviation[2],
                'mean_squared_deviation_green': mean_squared_deviation[3],
                'total_sum_of_squares': total_sum_of_squares,
                'fcsd_ir': fcsd[0],
                'fcsd_red_edge': fcsd[1],
                'fcsd_red': fcsd[2],
                'fcsd_green': fcsd[3]
            })

# The `grid_data` list now contains the calculated values for each block within the TIFF file bounds.

# Convert the list to a GeoDataFrame
result_gdf = gpd.GeoDataFrame(grid_data, crs=grid.crs)

# Save the results as a new shapefile
result_gdf.to_file(output_shapefile)

print(f"Grid shapefile with calculations saved as '{output_shapefile}'")

C:\Users\user\AppData\Local\Temp\ipykernel_13412\3251467657.py:35: RuntimeWarning: invalid value encountered in divide
  mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))
C:\Users\user\AppData\Local\Temp\ipykernel_13412\3251467657.py:35: RuntimeWarning: invalid value encountered in divide
  mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))
C:\Users\user\AppData\Local\Temp\ipykernel_13412\3251467657.py:35: RuntimeWarning: invalid value encountered in divide
  mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))
C:\Users\user\AppData\Local\Temp\ipykernel_13412\3251467657.py:35: RuntimeWarning: invalid value encountered in divide
  mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))
C:\Users\user\AppData\Local\Temp\ipykernel_13412\3251467657.py:35: RuntimeWarning: invalid value encountered in divide
  mean_squared_deviation = sum_of_squares / np.sum(masked_tile > 0, axis=(1, 2))


Grid shapefile with calculations saved as 'h:\Yehmh\DNDF\202404_DNDF\spectral_diversity\alpha_diversity_filtered4.shp'


In [4]:
import geopandas as gpd

# Read the shapefile
gdf = gpd.read_file('h:\\Yehmh\\DNDF\\202404_DNDF\\spectral_diversity\\alpha_diversity_filtered3.shp')
gdf = gpd.read_file('h:\\Yehmh\\DNDF\\202404_DNDF\\spectral_diversity\\north_grid.shp')

# Print the first few rows of the GeoDataFrame
print(gdf.head())

   FID                                           geometry
0    0  POLYGON ((293587.577 2611930.218, 293487.577 2...
1    1  POLYGON ((293587.577 2612030.218, 293487.577 2...
2    2  POLYGON ((293587.577 2612130.218, 293487.577 2...
3    3  POLYGON ((293587.577 2612230.218, 293487.577 2...
4    4  POLYGON ((293587.577 2612330.218, 293487.577 2...
